In [1]:
pip install dash pandas plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.1
    Uninstalling Flask-3.1.1:
      Successfully uninstalled Flask-3.1.1


In [2]:
import dash
from dash import dcc, html, Input, Output, dash_table
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime as dt

# Sample data - in a real scenario, you'd load this from a database or CSV
def create_sample_data():
    num_days = 365  # Number of days in the date range

    # Create the Date range
    dates = pd.date_range(start='2023-01-01', end='2023-12-31', freq='D')

    # Ensure 'Product' and 'Region' have the correct number of elements
    products = ['A', 'B', 'C'] * (num_days // 3 + 1)
    products = products[:num_days]  # Trim to num_days

    regions = ['North', 'South', 'East', 'West'] * (num_days // 4 + 1)
    regions = regions[:num_days]  # Trim to num_days

    data = {
        'Date': dates,
        'Product': products,
        'Region': regions,
        'Sales': [round(i * 10 + 100, 2) for i in range(num_days)],
        'Quantity': [i % 20 + 1 for i in range(num_days)],
        'Profit': [round(i * 2 + 20, 2) for i in range(num_days)]
    }

    df = pd.DataFrame(data)
    df['Month'] = df['Date'].dt.month_name()
    df['Quarter'] = 'Q' + df['Date'].dt.quarter.astype(str)
    return df

df = create_sample_data()



In [3]:


# Initialize the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    html.Div([
        html.H1("Sales Performance Dashboard", style={'text-align': 'center'}),

        # Filters Row
        html.Div([
            html.Div([
                html.Label("Date Range:"),
                dcc.DatePickerRange(
                    id='date-range',
                    start_date=df['Date'].min(),
                    end_date=df['Date'].max(),
                    display_format='YYYY-MM-DD'
                )
            ], className='three columns'),

            html.Div([
                html.Label("Region:"),
                dcc.Dropdown(
                    id='region-selector',
                    options=[{'label': r, 'value': r} for r in df['Region'].unique()],
                    value=['North', 'South', 'East', 'West'],
                    multi=True
                )
            ], className='three columns'),

            html.Div([
                html.Label("Product:"),
                dcc.Dropdown(
                    id='product-selector',
                    options=[{'label': p, 'value': p} for p in df['Product'].unique()],
                    value=['A', 'B', 'C'],
                    multi=True
                )
            ], className='three columns'),

            html.Div([
                html.Label("View by:"),
                dcc.RadioItems(
                    id='time-aggregation',
                    options=[
                        {'label': 'Daily', 'value': 'D'},
                        {'label': 'Weekly', 'value': 'W'},
                        {'label': 'Monthly', 'value': 'M'},
                        {'label': 'Quarterly', 'value': 'Q'}
                    ],
                    value='M',
                    inline=True
                )
            ], className='three columns')
        ], className='row', style={'margin-bottom': '20px'}),

        # KPI Cards Row
        html.Div([
            html.Div([
                html.Div([
                    html.H3(id='total-sales', children="0"),
                    html.P("Total Sales")
                ], className='kpi-card')
            ], className='three columns'),

            html.Div([
                html.Div([
                    html.H3(id='total-profit', children="0"),
                    html.P("Total Profit")
                ], className='kpi-card')
            ], className='three columns'),

            html.Div([
                html.Div([
                    html.H3(id='avg-sale', children="0"),
                    html.P("Average Sale")
                ], className='kpi-card')
            ], className='three columns'),

            html.Div([
                html.Div([
                    html.H3(id='total-transactions', children="0"),
                    html.P("Total Transactions")
                ], className='kpi-card')
            ], className='three columns')
        ], className='row', style={'margin-bottom': '20px'}),

        # Main Charts Row
        html.Div([
            html.Div([
                dcc.Graph(id='sales-trend')
            ], className='six columns'),

            html.Div([
                dcc.Graph(id='profit-by-product')
            ], className='six columns')
        ], className='row'),

        # Secondary Charts Row
        html.Div([
            html.Div([
                dcc.Graph(id='region-performance')
            ], className='six columns'),

            html.Div([
                dcc.Graph(id='sales-by-month')
            ], className='six columns')
        ], className='row'),

        # Data Table Row
        html.Div([
            html.H3("Detailed Data"),
            dash_table.DataTable(
                id='data-table',
                columns=[{"name": i, "id": i} for i in df.columns],
                page_size=10,
                style_table={'overflowX': 'auto'},
                style_cell={
                    'height': 'auto',
                    'minWidth': '100px', 'width': '100px', 'maxWidth': '180px',
                    'whiteSpace': 'normal'
                }
            )
        ], className='row', style={'margin-top': '20px'})
    ], className='container')
])

# Define callback functions
@app.callback(
    [Output('total-sales', 'children'),
     Output('total-profit', 'children'),
     Output('avg-sale', 'children'),
     Output('total-transactions', 'children'),
     Output('sales-trend', 'figure'),
     Output('profit-by-product', 'figure'),
     Output('region-performance', 'figure'),
     Output('sales-by-month', 'figure'),
     Output('data-table', 'data')],
    [Input('date-range', 'start_date'),
     Input('date-range', 'end_date'),
     Input('region-selector', 'value'),
     Input('product-selector', 'value'),
     Input('time-aggregation', 'value')]
)
def update_dashboard(start_date, end_date, regions, products, time_agg):
    # Filter data based on inputs
    filtered_df = df[
        (df['Date'] >= start_date) &
        (df['Date'] <= end_date) &
        (df['Region'].isin(regions)) &
        (df['Product'].isin(products))
    ]

    # Calculate KPIs
    total_sales = filtered_df['Sales'].sum()
    total_profit = filtered_df['Profit'].sum()
    avg_sale = filtered_df['Sales'].mean()
    total_transactions = filtered_df.shape[0]

    # Format KPI values
    total_sales_str = f"${total_sales:,.2f}"
    total_profit_str = f"${total_profit:,.2f}"
    avg_sale_str = f"${avg_sale:,.2f}"
    total_transactions_str = f"{total_transactions:,}"

    # Prepare data for time series
    if time_agg == 'D':
        time_group = pd.Grouper(key='Date', freq='D')
    elif time_agg == 'W':
        time_group = pd.Grouper(key='Date', freq='W-Mon')
    elif time_agg == 'M':
        time_group = pd.Grouper(key='Date', freq='M')
    elif time_agg == 'Q':
        time_group = pd.Grouper(key='Date', freq='Q')

    time_series_df = filtered_df.groupby(time_group).agg({
        'Sales': 'sum',
        'Profit': 'sum',
        'Quantity': 'sum'
    }).reset_index()

    # Create sales trend chart
    sales_trend_fig = px.line(
        time_series_df,
        x='Date',
        y='Sales',
        title='Sales Trend Over Time',
        labels={'Sales': 'Total Sales ($)'}
    )
    sales_trend_fig.update_layout(hovermode='x unified')

    # Create profit by product chart
    profit_by_product_fig = px.bar(
        filtered_df.groupby('Product').agg({'Profit': 'sum'}).reset_index(),
        x='Product',
        y='Profit',
        title='Profit by Product',
        color='Product',
        labels={'Profit': 'Total Profit ($)'}
    )

    # Create region performance chart
    region_performance_fig = px.pie(
        filtered_df.groupby('Region').agg({'Sales': 'sum'}).reset_index(),
        names='Region',
        values='Sales',
        title='Sales Distribution by Region',
        hole=0.4
    )

    # Create sales by month chart
    sales_by_month_fig = px.box(
        filtered_df,
        x='Month',
        y='Sales',
        title='Sales Distribution by Month',
        color='Month'
    )

    # Prepare data for table
    table_data = filtered_df.to_dict('records')

    return (
        total_sales_str,
        total_profit_str,
        avg_sale_str,
        total_transactions_str,
        sales_trend_fig,
        profit_by_product_fig,
        region_performance_fig,
        sales_by_month_fig,
        table_data
    )

# Add CSS styling
app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})

# Run the app
if __name__ == '__main__':
    app.run(debug=True) # Changed app.run_server to app.run


/usr/local/lib/python3.11/dist-packages/dash/resources.py:96: UserWarning:

You have set your config to `serve_locally=True` but A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plotly.com/external-resources



<IPython.core.display.Javascript object>